# 通信会社 ネットワーク品質 AI エージェント ハンズオン

このハンズオンでは、Snowflake の AI 機能を使って**エリアマスタ、通信品質データ、設備キャパシティデータ、気象データ**の整備から **「自然言語で質問できる AI エージェント作成」** までを一気通貫で構築します。

### 今日つくるもの

携帯電話通信会社のネットワーク品質データ（通信速度・遅延・接続成功率・トラフィック量など）を横断的に分析できる AI エージェントです。  
完成すると、例えばこんな質問に自然言語で答えてくれます：

> 「2025年の東京エリアの通信速度推移を月別で可視化して。気温との相関も見たい」  
> 「現在設備が逼迫しているエリアと、そのトラフィック推移を教えて」  
> 「関東地方のエリア一覧とそれぞれの特徴を教えて」  
> 「雨の日と晴れの日で通信品質にどんな違いがある？」

---

### ハンズオンの流れ（所要時間：約60分 ＋ 応用25分）

| Step | やること | 学べる Snowflake 機能 | 所要時間 |
|:----:|----------|----------------------|:--------:|
| 1 | AI でエリアマスタを整備 | AI_CLASSIFY / AI_COMPLETE | 10分 |
| 2 | 気象データを取り込む | Snowflake Marketplace | 10分 |
| 3 | エリアの意味検索を可能にする | Cortex Search Service | 10分 |
| 4 | データの意味を定義する | Semantic View（Cortex Analyst） | 20分 |
| 5 | AI エージェントを組み立てる | Cortex Agent | 15分 |
| 6 | 動かしてみよう！ | Cortex Agent UI | 5分 |
| 7 | 【応用】カスタムモデルで予兆検知 | Snowpark ML / Container Runtime / Model Registry | 25分 |

> **Note**: Step 7 は応用編です。時間に余裕がある場合にチャレンジしてください。Step 1〜6 だけでも十分にハンズオンは完結します。

## Step 0: 事前準備
使用するデータベース・スキーマ・ウェアハウスを設定します。

In [ ]:
USE DATABASE TELECOM_AI_HANDSON;
USE SCHEMA ANALYTICS;
USE WAREHOUSE COMPUTE_WH;

---
## Step 1: AI でエリアマスタを整備する

現在のエリアマスタ（`RAW.AREA_MASTER`）には、エリアID・エリア名・地方名しかありません。  
ここに Snowflake の **AI SQL 関数** を使って、2つの情報を自動で追加します。

### 追加する情報

#### 1. エリアタイプの自動分類 — `AI_CLASSIFY`

エリア名だけを見て、AI が自動的にエリアタイプを判定します。  
あらかじめ定義した **5つのタイプ** に分類します（MECE＝漏れなくダブりなく）。

| エリアタイプ | どんなエリア？ |
|-------------|---------------|
| 都心ビジネス | オフィス街、駅前商業地区、高層ビル密集地 |
| 商業・繁華街 | ショッピングモール、繁華街、娯楽施設集中地 |
| 住宅地 | マンション・戸建て住宅が中心の生活エリア |
| 観光・ウォーターフロント | 観光地、海辺、リゾート、歴史的エリア |
| 郊外・地方 | 工業地帯、農村部、山間地、人口密度が低いエリア |

#### 2. エリア特性説明文の自動生成 — `AI_COMPLETE`

エリア名と地方名から、そのエリアの **通信需要の特性や課題を説明する文章（約300文字）** を AI が自動で生成します。

> **ポイント**: `AI_CLASSIFY` はラベル＋説明文を渡すだけで分類してくれるので、従来のルールベースのロジックを書く必要がありません。

In [ ]:
SELECT * FROM TELECOM_AI_HANDSON.RAW.AREA_MASTER;

それでは実行しましょう。エリア数にもよりますが、**約1〜2分** で完了します。

In [ ]:
CREATE OR REPLACE TABLE TELECOM_AI_HANDSON.ANALYTICS.AI_AREA_MASTER AS
SELECT 
    AREA_ID,
    AREA_NAME,
    REGION_NAME,
    REGION_CODE,
    -- ① AI_CLASSIFY関数でエリアタイプを分類（MECE）
    AI_CLASSIFY(
        AREA_NAME,
        [
            {'label': '都心ビジネス', 'description': 'オフィス街、駅前商業地区、高層ビル密集地、ビジネス中心地'},
            {'label': '商業・繁華街', 'description': 'ショッピングモール、繁華街、娯楽施設集中地、歓楽街'},
            {'label': '住宅地', 'description': 'マンション・戸建て住宅が中心の生活エリア、ベッドタウン'},
            {'label': '観光・ウォーターフロント', 'description': '観光地、海辺、リゾート、歴史的エリア、臨海地区、湾岸'},
            {'label': '郊外・地方', 'description': '工業地帯、農村部、山間地、人口密度が低いエリア、地方都市郊外'}
        ],
        {'task_description': '日本の携帯電話通信エリア名から最も適切なエリアタイプに分類してください'}
    ):labels[0]::VARCHAR AS AREA_TYPE,
    -- ② AI_COMPLETE関数でエリア特性説明文生成
    TRIM(SNOWFLAKE.CORTEX.AI_COMPLETE(
        'llama3.1-405b',
        '以下の携帯電話通信エリアについて、通信需要の特性や課題を説明する文章（300文字程度）を日本語で1つ作成してください。
         エリア名: ' || AREA_NAME || '
         地方: ' || COALESCE(REGION_NAME, '不明') || '
         説明文:'
    )) AS AREA_DESCRIPTION,
    CURRENT_TIMESTAMP() AS CREATED_AT
FROM TELECOM_AI_HANDSON.RAW.AREA_MASTER;

### 結果を見てみましょう

AI が生成した `AREA_TYPE`（エリアタイプ）と `AREA_DESCRIPTION`（エリア特性説明文）を確認します。

In [ ]:
SELECT * FROM TELECOM_AI_HANDSON.ANALYTICS.AI_AREA_MASTER;

### エリアタイプ分類の精度を確認

5つのエリアタイプにバランスよく分類されているか確認しましょう。  
偏りがある場合は、`AI_CLASSIFY` に渡すエリアタイプの `description` を調整することで精度を改善できます。

In [ ]:
SELECT 
    AREA_TYPE,
    COUNT(*) AS AREA_COUNT
FROM TELECOM_AI_HANDSON.ANALYTICS.AI_AREA_MASTER
GROUP BY AREA_TYPE
ORDER BY AREA_COUNT DESC;

---
## Step 2: 気象データを取り込む（Snowflake Marketplace）

通信品質は天候に大きく左右されます。  
雨天時は電波の減衰が増し通信品質が低下、台風時はさらに顕著 — こうした **天候×通信品質の分析** を可能にするため、  
Snowflake Marketplace の **気象データ** を取り込みます。

### データソース

**Weather Source LLC: Frostbyte** が提供するグローバル気象データを使用します。  
日本（JP）の日次気象データを 2019年〜 取得できます。

### なぜテーブルを作るのか？

元データは華氏・インチ単位で、グローバルデータが含まれています。  
通信品質データと結合するには **日本のデータのみを抽出し、メートル法に変換して日付ごとに1行** にする必要があるため、  
全国平均に集約するテーブルを作成します。

| 集約カラム | 計算方法 | 説明 |
|------------|----------|------|
| 平均気温 | AVG | 全国の観測所の平均（℃に変換） |
| 最高気温 | MAX | 全国で最も高かった気温（℃に変換） |
| 最低気温 | MIN | 全国で最も低かった気温（℃に変換） |
| 降水量 | AVG | 全国平均の降水量（mmに変換） |
| 湿度 | AVG | 全国平均の湿度 |
| 日照時間 | AVG | 全国平均の日射量 |
| 降雪量 | SUM | 全国の降雪量合計（cmに変換） |
| 風速 | AVG | 全国平均の風速（km/hに変換） |

In [ ]:
CREATE OR REPLACE TABLE TELECOM_AI_HANDSON.ANALYTICS.DAILY_WEATHER AS
SELECT
    DATE_VALID_STD                                                      AS WEATHER_DATE,
    ROUND(AVG((AVG_TEMPERATURE_AIR_2M_F - 32) * 5/9), 1)                AS AVG_TEMPERATURE,
    ROUND(MAX((MAX_TEMPERATURE_AIR_2M_F - 32) * 5/9), 1)                AS MAX_TEMPERATURE,
    ROUND(MIN((MIN_TEMPERATURE_AIR_2M_F - 32) * 5/9), 1)                AS MIN_TEMPERATURE,
    ROUND(AVG(TOT_PRECIPITATION_IN * 25.4), 1)                          AS AVG_RAINFALL,
    ROUND(AVG(AVG_HUMIDITY_RELATIVE_2M_PCT), 1)                         AS AVG_HUMIDITY,
    ROUND(AVG(AVG_RADIATION_SOLAR_TOTAL_WPM2), 2)                       AS AVG_SUNLIGHT_HOURS,
    ROUND(SUM(TOT_SNOWFALL_IN * 25.4), 1)                               AS TOTAL_SNOWFALL,
    ROUND(AVG(AVG_WIND_SPEED_10M_MPH * 1.60934), 1)                     AS AVG_WIND_SPEED
FROM WEATHER_SOURCE_LLC_FROSTBYTE.ONPOINT_ID.HISTORY_DAY
WHERE COUNTRY = 'JP'
GROUP BY DATE_VALID_STD
ORDER BY DATE_VALID_STD;

In [ ]:
SELECT * FROM TELECOM_AI_HANDSON.ANALYTICS.DAILY_WEATHER;

---
## Step 3: エリアの意味検索を可能にする（Cortex Search）

Step 1 で作った `AI_AREA_MASTER` に対して、**Cortex Search Service** を作成します。

### Cortex Search とは？

テキストの **意味** を理解して検索するサービスです。  
キーワードの完全一致ではなく、「こういう意味のものを探して」という検索ができます。

例えば「トラフィックが多い都市部のエリア」と聞くと、東京都心や大阪梅田などのエリアがヒットします。

### 仕組み

エリア名・地方名・エリアタイプ・AI説明文を **1つのテキストに結合** し、  
Snowflake の Embedding モデル（`snowflake-arctic-embed-l-v2.0`）でベクトル化して検索可能にします。

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE TELECOM_AI_HANDSON.ANALYTICS.AREA_SEARCH_SERVICE
    ON SEARCH_TEXT
    ATTRIBUTES AREA_ID, AREA_NAME, REGION_NAME, AREA_TYPE, AREA_DESCRIPTION
    WAREHOUSE = COMPUTE_WH
    TARGET_LAG = '1 day'
    EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
AS (
    SELECT
        AREA_ID,
        AREA_NAME,
        REGION_NAME,
        REGION_CODE,
        AREA_TYPE,
        AREA_DESCRIPTION,
        CONCAT(
            COALESCE(AREA_NAME, ''), ' ',
            COALESCE(REGION_NAME, ''), ' ',
            COALESCE(AREA_TYPE, ''), ' ',
            COALESCE(AREA_DESCRIPTION, '')
        ) AS SEARCH_TEXT
    FROM TELECOM_AI_HANDSON.ANALYTICS.AI_AREA_MASTER
);

---
## Step 4: データの意味を定義する（セマンティックビュー）

ここがこのハンズオンの **キモ** です。

Cortex Analyst に「通信速度を教えて」と聞いたとき、どのテーブルの・どのカラムを・どう集計すればよいかを  
AI が理解できるように、**データの意味（セマンティクス）** を定義します。

### セマンティックビューで定義すること

| 定義項目 | 何を定義する？ | 具体例 |
|----------|---------------|--------|
| **テーブル** | 分析対象のテーブルとその説明 | NETWORK_QUALITY, EQUIPMENT_STATUS, WEATHER |
| **リレーションシップ** | テーブル間の結合条件 | 計測日 = 気象観測日 |
| **ファクト** | 集計対象の数値カラム | 通信速度、遅延、トラフィック量、気温 |
| **ディメンション** | 分析の切り口となる属性 | エリア名、地方、日付 |
| **メトリクス** | よく使う集計パターン | 平均ダウンロード速度＝AVG(下り速度) |
| **シノニム** | 日本語の別名・同義語 | 「通信速度」「下り速度」「ダウンロード速度」 |

### 3つのテーブルの関係

```
              AREA_ID              MEASURE_DATE = WEATHER_DATE
NETWORK_QUALITY ───────▶ EQUIPMENT_STATUS    NETWORK_QUALITY ────────────────▶ WEATHER
(通信品質)                (設備状況)           (通信品質)                        (気象)
```

> **ポイント**: シノニムを豊富に定義しておくと、「通信速度」でも「ダウンロード速度」でも「下り」でも同じカラムを参照してくれます。

In [ ]:
CREATE OR REPLACE SEMANTIC VIEW TELECOM_AI_HANDSON.ANALYTICS.TELECOM_ANALYST

    -- ========================================
    -- テーブル定義
    -- ========================================
    tables (
        NETWORK_QUALITY AS TELECOM_AI_HANDSON.ANALYTICS.NETWORK_QUALITY
            PRIMARY KEY (AREA_ID, MEASURE_DATE)
            WITH SYNONYMS = ('通信品質', 'ネットワーク品質', '品質データ', '通信データ', 'NW品質', '通信速度データ', 'KPI')
            COMMENT = '日次ネットワーク品質データ。2022-01-01〜2026-02-24のエリア別日次通信速度・遅延・接続成功率・トラフィック量を含む。',

        EQUIPMENT_STATUS AS TELECOM_AI_HANDSON.ANALYTICS.EQUIPMENT_STATUS
            PRIMARY KEY (AREA_ID)
            WITH SYNONYMS = ('設備', '設備状況', '設備データ', '基地局', 'キャパシティ', '設備キャパシティ', '負荷')
            COMMENT = '設備キャパシティスナップショット（2026-02-24時点）。エリア別の基地局数・最大容量・負荷率・設備ステータスを含む。',

        WEATHER AS TELECOM_AI_HANDSON.ANALYTICS.DAILY_WEATHER
            PRIMARY KEY (WEATHER_DATE)
            WITH SYNONYMS = ('気象', '天気', '気象データ', '天気データ', '気温', '降水量', '天候', '気象庁', '気象情報')
            COMMENT = '全国日次平均気象データ（2015年〜）。気象庁の全観測所データを日別に集約した全国平均の気温・降水量・湿度・日照時間・降雪量・風速。'
    )

    -- ========================================
    -- リレーションシップ定義
    -- ========================================
    relationships (
        NQ_EQUIPMENT AS NETWORK_QUALITY(AREA_ID) REFERENCES EQUIPMENT_STATUS(AREA_ID),
        NQ_WEATHER AS NETWORK_QUALITY(MEASURE_DATE) REFERENCES WEATHER(WEATHER_DATE)
    )

    -- ========================================
    -- ファクト定義（数値データ）
    -- ========================================
    facts (
        -- 通信品質ファクト
        NETWORK_QUALITY.download_speed_fact AS AVG_DOWNLOAD_MBPS
            COMMENT = '平均ダウンロード速度（Mbps）。下り通信速度。',
        NETWORK_QUALITY.upload_speed_fact AS AVG_UPLOAD_MBPS
            COMMENT = '平均アップロード速度（Mbps）。上り通信速度。',
        NETWORK_QUALITY.latency_fact AS AVG_LATENCY_MS
            COMMENT = '平均遅延（ミリ秒）。レイテンシ。',
        NETWORK_QUALITY.connection_success_fact AS CONNECTION_SUCCESS_RATE
            COMMENT = '接続成功率（%）。100%に近いほど良好。',
        NETWORK_QUALITY.call_drop_fact AS CALL_DROP_RATE
            COMMENT = '切断率（%）。0%に近いほど良好。',
        NETWORK_QUALITY.packet_loss_fact AS PACKET_LOSS_RATE
            COMMENT = 'パケットロス率（%）。0%に近いほど良好。',
        NETWORK_QUALITY.traffic_fact AS TOTAL_TRAFFIC_GB
            COMMENT = '日次総トラフィック量（GB）',

        -- 設備ファクト
        EQUIPMENT_STATUS.base_station_count_fact AS BASE_STATION_COUNT
            COMMENT = '基地局数（局）',
        EQUIPMENT_STATUS.max_capacity_fact AS MAX_CAPACITY_GBPS
            COMMENT = '最大通信容量（Gbps）',
        EQUIPMENT_STATUS.current_load_fact AS CURRENT_LOAD_PCT
            COMMENT = '現在の設備負荷率（%）。100%に近いほど逼迫。',

        -- 気象ファクト
        WEATHER.avg_temperature_fact AS AVG_TEMPERATURE
            COMMENT = '全国日平均気温（℃）。全観測所の平均。',
        WEATHER.max_temperature_fact AS MAX_TEMPERATURE
            COMMENT = '全国日最高気温（℃）。全観測所の最大値。',
        WEATHER.min_temperature_fact AS MIN_TEMPERATURE
            COMMENT = '全国日最低気温（℃）。全観測所の最小値。',
        WEATHER.avg_rainfall_fact AS AVG_RAINFALL
            COMMENT = '全国日平均降水量（mm）',
        WEATHER.avg_humidity_fact AS AVG_HUMIDITY
            COMMENT = '全国日平均湿度（%）',
        WEATHER.avg_sunlight_hours_fact AS AVG_SUNLIGHT_HOURS
            COMMENT = '全国日平均日照時間（時間）',
        WEATHER.total_snowfall_fact AS TOTAL_SNOWFALL
            COMMENT = '全国日降雪量合計（cm）',
        WEATHER.avg_wind_speed_fact AS AVG_WIND_SPEED
            COMMENT = '全国日平均風速（km/h）'
    )

    -- ========================================
    -- ディメンション定義（属性データ）
    -- ========================================
    dimensions (
        -- ========== NETWORK_QUALITY ディメンション ==========
        NETWORK_QUALITY.area_id AS AREA_ID
            WITH SYNONYMS = ('エリアID', 'エリアコード', '基地局エリア', 'エリア識別子')
            COMMENT = 'エリア識別コード（例：AREA_KT_001）',
        NETWORK_QUALITY.area_name AS AREA_NAME
            WITH SYNONYMS = ('エリア名', 'エリア', '地域名', '地区名', '基地局エリア名')
            COMMENT = '通信エリア名。東京都心北部エリア、大阪梅田エリアなどを含む。',
        NETWORK_QUALITY.region_name AS REGION_NAME
            WITH SYNONYMS = ('地方', '地方名', 'リージョン', '地域')
            COMMENT = '地方名（関東、近畿、東海、九州）',
        NETWORK_QUALITY.measure_date AS MEASURE_DATE
            WITH SYNONYMS = ('計測日', '測定日', '日付', '対象日')
            COMMENT = '通信品質を計測した日付（DATE型、2022-01-01〜2026-02-24）',
        NETWORK_QUALITY.measure_year AS YEAR(MEASURE_DATE)
            WITH SYNONYMS = ('計測年', '年', '対象年')
            COMMENT = '計測年（2022〜2026）',
        NETWORK_QUALITY.measure_month AS MONTH(MEASURE_DATE)
            WITH SYNONYMS = ('計測月', '月', '対象月')
            COMMENT = '計測月（1〜12）',

        -- ========== EQUIPMENT_STATUS ディメンション ==========
        EQUIPMENT_STATUS.area_id AS AREA_ID
            WITH SYNONYMS = ('エリアID', 'エリアコード')
            COMMENT = 'エリア識別コード',
        EQUIPMENT_STATUS.area_name AS AREA_NAME
            WITH SYNONYMS = ('エリア名', 'エリア')
            COMMENT = 'エリア名',
        EQUIPMENT_STATUS.region_name AS REGION_NAME
            WITH SYNONYMS = ('地方', '地方名')
            COMMENT = '地方名',
        EQUIPMENT_STATUS.snapshot_date AS SNAPSHOT_DATE
            WITH SYNONYMS = ('スナップショット日', '基準日', '設備確認日')
            COMMENT = '設備データの取得日（2026-02-24）',
        EQUIPMENT_STATUS.equipment_status AS EQUIPMENT_STATUS
            WITH SYNONYMS = ('設備ステータス', '設備状態', '設備区分', 'ステータス', '負荷状態')
            COMMENT = '設備の負荷状態を表すステータス（正常 / 注意 / 逼迫）',

        -- ========== WEATHER ディメンション ==========
        WEATHER.weather_date AS WEATHER_DATE
            WITH SYNONYMS = ('観測日', '気象日', '天気の日付', '気象観測日')
            COMMENT = '気象データの観測日（DATE型、2015年〜）。NETWORK_QUALITYのMEASURE_DATEとリレーションで紐づく。',
        WEATHER.weather_year AS YEAR(WEATHER_DATE)
            WITH SYNONYMS = ('観測年', '気象年')
            COMMENT = '気象データの観測年',
        WEATHER.weather_month AS MONTH(WEATHER_DATE)
            WITH SYNONYMS = ('観測月', '気象月')
            COMMENT = '気象データの観測月（1〜12）'
    )

    -- ========================================
    -- メトリクス定義（集計関数）
    -- ========================================
    metrics (
        -- 通信品質メトリクス
        NETWORK_QUALITY.avg_download_speed AS AVG(NETWORK_QUALITY.download_speed_fact)
            WITH SYNONYMS = ('平均ダウンロード速度', '平均下り速度', '下り平均', '平均通信速度')
            COMMENT = 'ダウンロード速度の平均（Mbps）',
        NETWORK_QUALITY.avg_upload_speed AS AVG(NETWORK_QUALITY.upload_speed_fact)
            WITH SYNONYMS = ('平均アップロード速度', '平均上り速度', '上り平均')
            COMMENT = 'アップロード速度の平均（Mbps）',
        NETWORK_QUALITY.avg_latency AS AVG(NETWORK_QUALITY.latency_fact)
            WITH SYNONYMS = ('平均遅延', '平均レイテンシ', 'レイテンシ平均')
            COMMENT = '遅延の平均（ms）',
        NETWORK_QUALITY.avg_connection_success AS AVG(NETWORK_QUALITY.connection_success_fact)
            WITH SYNONYMS = ('平均接続成功率', '接続成功率平均', '接続率')
            COMMENT = '接続成功率の平均（%）',
        NETWORK_QUALITY.avg_call_drop AS AVG(NETWORK_QUALITY.call_drop_fact)
            WITH SYNONYMS = ('平均切断率', '切断率平均', '通話切断率')
            COMMENT = '切断率の平均（%）',
        NETWORK_QUALITY.avg_packet_loss AS AVG(NETWORK_QUALITY.packet_loss_fact)
            WITH SYNONYMS = ('平均パケットロス率', 'パケロス平均', 'ロス率')
            COMMENT = 'パケットロス率の平均（%）',
        NETWORK_QUALITY.total_traffic AS SUM(NETWORK_QUALITY.traffic_fact)
            WITH SYNONYMS = ('総トラフィック', '合計トラフィック', 'トラフィック合計', '総通信量')
            COMMENT = 'トラフィック量の合計（GB）',
        NETWORK_QUALITY.avg_daily_traffic AS AVG(NETWORK_QUALITY.traffic_fact)
            WITH SYNONYMS = ('日平均トラフィック', '日次平均トラフィック', '平均日次通信量')
            COMMENT = '日次トラフィック量の平均（GB）',

        -- 設備メトリクス
        EQUIPMENT_STATUS.total_base_stations AS SUM(EQUIPMENT_STATUS.base_station_count_fact)
            WITH SYNONYMS = ('総基地局数', '合計基地局数', '基地局数合計')
            COMMENT = '基地局数の合計（局）',
        EQUIPMENT_STATUS.total_capacity AS SUM(EQUIPMENT_STATUS.max_capacity_fact)
            WITH SYNONYMS = ('総容量', '合計容量', '最大容量合計')
            COMMENT = '最大通信容量の合計（Gbps）',
        EQUIPMENT_STATUS.avg_load AS AVG(EQUIPMENT_STATUS.current_load_fact)
            WITH SYNONYMS = ('平均負荷率', '負荷率平均', '平均設備負荷')
            COMMENT = '設備負荷率の平均（%）',

        -- 気象メトリクス
        WEATHER.period_avg_temperature AS AVG(WEATHER.avg_temperature_fact)
            WITH SYNONYMS = ('平均気温', '気温平均', '平均温度')
            COMMENT = '期間内の全国平均気温（℃）',
        WEATHER.period_max_temperature AS MAX(WEATHER.max_temperature_fact)
            WITH SYNONYMS = ('最高気温', '最高温度', '期間最高気温')
            COMMENT = '期間中の全国最高気温（℃）',
        WEATHER.period_min_temperature AS MIN(WEATHER.min_temperature_fact)
            WITH SYNONYMS = ('最低気温', '最低温度', '期間最低気温')
            COMMENT = '期間中の全国最低気温（℃）',
        WEATHER.period_avg_rainfall AS AVG(WEATHER.avg_rainfall_fact)
            WITH SYNONYMS = ('平均降水量', '降水量平均')
            COMMENT = '期間内の全国平均降水量（mm）',
        WEATHER.period_avg_humidity AS AVG(WEATHER.avg_humidity_fact)
            WITH SYNONYMS = ('平均湿度', '湿度平均')
            COMMENT = '期間内の全国平均湿度（%）',
        WEATHER.period_avg_sunlight AS AVG(WEATHER.avg_sunlight_hours_fact)
            WITH SYNONYMS = ('平均日照時間', '日照時間平均')
            COMMENT = '期間内の全国平均日照時間（時間）',
        WEATHER.period_total_snowfall AS SUM(WEATHER.total_snowfall_fact)
            WITH SYNONYMS = ('総降雪量', '合計降雪量', '累積降雪量')
            COMMENT = '期間内の全国降雪量合計（cm）',
        WEATHER.period_avg_wind_speed AS AVG(WEATHER.avg_wind_speed_fact)
            WITH SYNONYMS = ('平均風速', '風速平均')
            COMMENT = '期間内の全国平均風速（km/h）'
    )

    COMMENT = '携帯電話通信会社のネットワーク品質・設備・気象分析のためのセマンティックビュー。関東・近畿・東海・九州の50エリアの通信品質データを含む。2022年〜の日次通信品質データ（下り/上り速度、遅延、接続成功率、切断率、パケットロス率、トラフィック量）、2026-02-24時点の設備キャパシティスナップショット（基地局数、最大容量、負荷率）、および気象庁の全国日次平均気象データ（気温・降水量・湿度・日照時間・降雪量等）を統合。計測日と気象観測日がリレーションで紐づき、天候と通信品質の相関分析が可能。';

### セマンティックビューの確認

正しく定義できたか確認しましょう。テーブル・ファクト・ディメンション・メトリクスの一覧が表示されます。

In [ ]:
DESCRIBE SEMANTIC VIEW TELECOM_ANALYST;

---
## Step 5: AI エージェントを組み立てる（Cortex Agent）

いよいよエージェントを組み立てます。  
ここまで作ってきた **Cortex Search** と **Cortex Analyst** を組み合わせて、  
1つの **AI エージェント** にまとめます。

### エージェントの仕組み

```
ユーザーの質問
     │
     ▼
┌─────────────────────────┐
│   Telecom Agent        │
│  （オーケストレーター）   │
│                        │
│  質問の意図を判断して      │
│  適切なツールを選択        │
└────┬──────────┬─────────┘
     │          │
     ▼          ▼
┌─────────┐ ┌──────────────┐
│ Cortex  │ │   Cortex     │
│ Search  │ │  Analyst     │
│         │ │              │
│エリア情報│  │通信品質・設備・│
│の意味検索│ │気象の数値分析  │
└─────────┘ └──────────────┘
```

### ツールの使い分け

| 質問の種類 | 使われるツール | 例 |
|-----------|---------------|----|
| エリアの特徴・情報を知りたい | **Cortex Search** | 「関東のエリア一覧を教えて」 |
| 数値の集計・分析をしたい | **Cortex Analyst** | 「2025年の月別通信速度は？」 |
| 両方が必要 | **Search → Analyst** | 「逼迫エリアのトラフィック推移は？」 |

In [ ]:
CREATE OR REPLACE AGENT TELECOM_AI_HANDSON.ANALYTICS.TELECOM_AGENT
    COMMENT = '携帯電話通信会社のネットワーク品質・設備・気象分析とエリア検索を行うエージェント'
FROM SPECIFICATION $$
{
  "models": {
    "orchestration": ""
  },
  "instructions": {
    "response": "あなたは携帯電話通信会社のネットワーク品質アナリストです。ユーザーからの質問に対して、通信品質データ（下り/上り速度、遅延、接続成功率、切断率、パケットロス率、トラフィック量）、設備キャパシティデータ（基地局数、最大容量、負荷率）、気象データを活用して回答してください。数値データは適切にフォーマットして表示してください（例：速度はMbps表記、トラフィックはGB表記、比率は%表記）。エリア名は正確に記載してください。分析結果には根拠となるデータを含めてください。日本語で丁寧に回答してください。可能であれば視覚化を提供してください。時系列のトレンドは線グラフをデフォルトとし、エリア比較は棒グラフを使用してください。天候と通信品質の関係（雨天時の品質低下、気温とトラフィック量の相関など）にも注意して分析してください。",
    "orchestration": "エリアの詳細情報（エリア名、地方、エリアタイプ、AI説明文など）を検索する場合はcortex searchを使用し、通信品質・設備・気象の数値分析にはcortex analystを使用してください。複合的な質問の場合は、まずcortex searchでエリアを特定し、その結果をcortex analystに渡して分析してください。気温や天候と通信品質の関係を聞かれた場合は、cortex analystでネットワーク品質データと気象データを組み合わせて分析してください。",
    "sample_questions": [
      {
        "question": "2025年の東京都心エリアの下り通信速度推移を月別で可視化してください。気温との相関も見たいです。"
      },
      {
        "question": "トラフィック量が最も多い月はいつですか？どのエリアが特にトラフィックが集中しますか？"
      },
      {
        "question": "現在設備が逼迫しているエリアを教えてください。それぞれのエリアの過去のトラフィック推移も見せてください。"
      },
      {
        "question": "関東地方のエリア一覧とそれぞれの特徴を教えてください。"
      },
      {
        "question": "2024年と2025年でトラフィックが大きく増加したエリアトップ5を教えてください。"
      }
    ]
  },
  "tools": [
    {
      "tool_spec": {
        "type": "cortex_analyst_text_to_sql",
        "name": "TELECOM_ANALYST",
        "description": "携帯電話通信会社のネットワーク品質データ、設備キャパシティデータ、全国日次気象データをクエリして分析します。下り/上り通信速度、遅延、接続成功率、切断率、パケットロス率、トラフィック量の集計や、気温・降水量と通信品質の相関分析に使用します。2022年〜2026年2月の日次通信品質データ、2026年2月時点の設備キャパシティスナップショット、2015年〜の全国平均気象データを含みます。"
      }
    },
    {
      "tool_spec": {
        "type": "cortex_search",
        "name": "AREA_SEARCH",
        "description": "携帯電話通信エリアのマスタからエリア情報を検索します。エリア名、地方名、AIが分類したエリアタイプ（都心ビジネス、商業・繁華街、住宅地、観光・ウォーターフロント、郊外・地方）、AIが生成したエリア特性説明文などのテキスト検索に使用します。"
      }
    }
  ],
  "tool_resources": {
    "TELECOM_ANALYST": {
      "semantic_view": "TELECOM_AI_HANDSON.ANALYTICS.TELECOM_ANALYST"
    },
    "AREA_SEARCH": {
      "name": "TELECOM_AI_HANDSON.ANALYTICS.AREA_SEARCH_SERVICE",
      "max_results": 10
    }
  }
}
$$;

### エージェントの確認

In [ ]:
SHOW AGENTS IN SCHEMA TELECOM_AI_HANDSON.ANALYTICS;

---
## Step 6: 動かしてみよう！

エージェントが完成しました！Snowsight の **Cortex Agent UI** から以下のような質問を試してみましょう。

### 試してみたい質問例

**通信品質分析**
- 「2025年の東京都心エリアの下り通信速度推移を月別で可視化してください。気温との相関も見たいです。」
- 「2024年と2025年でトラフィックが大きく増加したエリアトップ5を教えてください。」

**天候×通信品質**
- 「降水量が多い日と少ない日で通信速度にどんな違いがありますか？」
- 「台風シーズン（8-9月）の通信品質への影響を分析してください。」

**設備管理**
- 「現在設備が逼迫しているエリアを教えてください。それぞれの過去のトラフィック推移も見せてください。」
- 「設備負荷率が注意レベルのエリアはどこですか？」

**エリア検索**
- 「関東地方のエリア一覧とそれぞれの特徴を教えてください。」
- 「観光地エリアの通信品質の傾向を教えてください。」

**Web検索**
- 「2026年に予定されている5G展開計画について教えてください。」
- 「最近の通信障害ニュースはありますか？」

---
## Step 7【応用編】: カスタムモデルで予兆検知（Snowpark ML / Container Runtime）

> **前提条件**: この Step は Snowflake Notebook の **Container Runtime** 上で実行します。  
> `setup_database.sql` の STEP 12 でコンピュートプールの作成と Notebook への割り当てが完了しています。  
> Snowsight の Notebook 設定で **Container Runtime が有効** になっていることを確認してください。  
> scikit-learn / XGBoost は Container Runtime に **プリインストール済み** のため、追加インストールは不要です。

### この Step でやること

1. **IsolationForest** で切断率の異常検知（教師なし）
2. **XGBRegressor** でトラフィック量の予測（教師あり）
3. 両モデルを **Model Registry** に登録し、推論を実行

In [ ]:
# Container Runtime にプリインストール済みのパッケージを確認
# scikit-learn と xgboost が含まれていることを確認します
!pip freeze | grep -E "scikit-learn|xgboost"

In [ ]:
# ============================================================
# データ取得: 通信品質 + 気象データを Pandas に読み込む
# ============================================================
import pandas as pd
from snowflake.snowpark.context import get_active_session

session = get_active_session()

# 全国平均の日次通信品質 + 気象データを結合して取得
df = session.sql("""
    SELECT
        nq.MEASURE_DATE,
        AVG(nq.CALL_DROP_RATE)          AS CALL_DROP_RATE,
        AVG(nq.TOTAL_TRAFFIC_GB)        AS TOTAL_TRAFFIC_GB,
        AVG(nq.AVG_DOWNLOAD_MBPS)       AS AVG_DOWNLOAD_MBPS,
        AVG(nq.AVG_LATENCY_MS)          AS AVG_LATENCY_MS,
        AVG(nq.PACKET_LOSS_RATE)        AS PACKET_LOSS_RATE,
        AVG(w.AVG_TEMPERATURE)          AS AVG_TEMPERATURE_C,
        AVG(w.AVG_RAINFALL)             AS PRECIPITATION_MM,
        AVG(w.AVG_WIND_SPEED)           AS WIND_SPEED_KMH
    FROM TELECOM_AI_HANDSON.ANALYTICS.NETWORK_QUALITY nq
    LEFT JOIN TELECOM_AI_HANDSON.ANALYTICS.DAILY_WEATHER w
        ON nq.MEASURE_DATE = w.WEATHER_DATE
    GROUP BY nq.MEASURE_DATE
    ORDER BY nq.MEASURE_DATE
""").to_pandas()

# 日付型変換・欠損値除去
df["MEASURE_DATE"] = pd.to_datetime(df["MEASURE_DATE"])
df = df.dropna()

# 訓練 / テスト分割（2025年以降をテスト）
train = df[df["MEASURE_DATE"] < "2025-01-01"].copy()
test  = df[df["MEASURE_DATE"] >= "2025-01-01"].copy()

print(f"訓練データ: {len(train)} 日, テストデータ: {len(test)} 日")
print(f"期間: {train['MEASURE_DATE'].min().date()} ~ {test['MEASURE_DATE'].max().date()}")
train.head()

### パート A: IsolationForest で異常検知

**IsolationForest** は、教師なし学習の異常検知アルゴリズムです。  
データポイントを「孤立させるのに必要な分割回数」で異常度を測定します。  
異常なデータは少ない分割で孤立するため、**異常スコアが高く** なります。

#### IsolationForest の特徴

| 項目 | 内容 |
|---|---|
| アルゴリズム | ランダムフォレストベースの孤立検知（教師なし） |
| ハイパーパラメータ | `n_estimators`（木の数）, `contamination`（異常割合）, `max_features`（特徴量数）等 |
| 特徴量 | 自由に設計可能（ラグ特徴量・移動平均等も組み合わせられる） |
| 出力 | 異常スコア（連続値） + 異常ラベル |

In [ ]:
# ============================================================
# IsolationForest: 訓練 → Model Registry に登録
# ============================================================
from sklearn.ensemble import IsolationForest
from snowflake.ml.registry import Registry

# --- 特徴量 ---
FEATURE_COLS = [
    "CALL_DROP_RATE", "AVG_DOWNLOAD_MBPS", "AVG_LATENCY_MS",
    "PACKET_LOSS_RATE", "PRECIPITATION_MM", "WIND_SPEED_KMH"
]

# --- モデル訓練 ---
# contamination: 異常と見なすデータの割合（5%）
# n_estimators: 決定木の本数（多いほど安定）
iso_model = IsolationForest(
    n_estimators=200,
    contamination=0.05,
    max_features=0.8,
    random_state=42
)
iso_model.fit(train[FEATURE_COLS])

# --- 訓練データでの異常検知結果を確認 ---
train_scores = iso_model.decision_function(train[FEATURE_COLS])
train_pred   = iso_model.predict(train[FEATURE_COLS])
print(f"訓練データの異常件数: {(train_pred == -1).sum()} / {len(train_pred)}")

# --- Model Registry に登録 ---
# Note: 登録時に UserWarning が数件表示されますが、いずれもエラーではなく正常動作です。
#   - relax_version: 依存ライブラリのバージョン制約が緩和される旨の通知（デフォルト動作）
#   - sample input: 入出力スキーマ推論に先頭100行のみ使用する旨の通知（全行は不要）
# target_platforms=["WAREHOUSE"] でウェアハウスからの推論を有効化
reg = Registry(session=session, database_name="TELECOM_AI_HANDSON", schema_name="ANALYTICS")

# 既存モデルがあれば削除して再登録（冪等性を確保）
try:
    reg.delete_model("NQ_ISOLATION_FOREST")
    print("既存モデル NQ_ISOLATION_FOREST を削除しました")
except Exception:
    pass

iso_mv = reg.log_model(
    model=iso_model,
    model_name="NQ_ISOLATION_FOREST",
    version_name="v1",
    conda_dependencies=["scikit-learn"],
    comment="IsolationForest: 通信品質の異常検知モデル（切断率+気象特徴量）",
    sample_input_data=train[FEATURE_COLS],
    target_platforms=["WAREHOUSE"],
    metrics={
        "n_estimators": 200,
        "contamination": 0.05,
        "train_anomaly_ratio": float((train_pred == -1).mean())
    }
)

print(f"✅ Model Registry に登録完了: NQ_ISOLATION_FOREST v1")

In [ ]:
# ============================================================
# IsolationForest: テストデータで推論 → 結果を確認
# ============================================================
import numpy as np

# --- テストデータで推論（ローカルの iso_model を直接使用）---
test_pred   = iso_model.predict(test[FEATURE_COLS])
test_scores = iso_model.decision_function(test[FEATURE_COLS])

test["ISO_ANOMALY"]       = (test_pred == -1)
test["ISO_ANOMALY_SCORE"] = test_scores

anomaly_days = test[test["ISO_ANOMALY"]]
print(f"テストデータの異常検知結果: {len(anomaly_days)} 日 / {len(test)} 日 を異常と判定")
print(f"異常日の切断率: 平均 {anomaly_days['CALL_DROP_RATE'].mean():.3f}, "
      f"最大 {anomaly_days['CALL_DROP_RATE'].max():.3f}")
print(f"正常日の切断率: 平均 {test[~test['ISO_ANOMALY']]['CALL_DROP_RATE'].mean():.3f}")

# --- Registry 経由の推論（mv.run はウェアハウスで実行）---
print("\n--- Registry 経由の推論（mv.run → Warehouse）---")
iso_mv_loaded = reg.get_model("NQ_ISOLATION_FOREST").version("v1")
# Snowpark DataFrame に変換して渡す
test_sp = session.create_dataframe(test[FEATURE_COLS].head(5))
registry_pred = iso_mv_loaded.run(test_sp, function_name="predict")
registry_pred.show()

# --- 異常日の上位10件を表示 ---
anomaly_days[["MEASURE_DATE", "CALL_DROP_RATE", "PRECIPITATION_MM",
              "WIND_SPEED_KMH", "ISO_ANOMALY_SCORE"]].sort_values(
    "ISO_ANOMALY_SCORE").head(10)

#### 結果の読み方

**1. 異常検知のサマリー**

```
テストデータの異常検知結果: 57 日 / 420 日 を異常と判定
異常日の切断率: 平均 0.298, 最大 0.544
正常日の切断率: 平均 0.313
```

- テストデータ 420 日のうち **57 日（約13.6%）** を異常と判定しました。
- `contamination=0.05`（5%）で訓練しましたが、テストデータでは異常が多めに検出されています。  
  これは **テスト期間（2025年〜）のデータ傾向が訓練期間と異なる** ことを示唆しています。
- 異常日の切断率の平均（0.298）が正常日（0.313）より **低い** 点に注目してください。  
  IsolationForest は切断率だけでなく **6つの特徴量の組み合わせ** で判定するため、  
  「切断率は低いが、降水量や風速が極端な日」なども異常として検出されます。

**2. Registry 経由の推論**

```
output_feature_0:  1 = 正常, -1 = 異常
```

- Model Registry に登録したモデルを `mv.run()` で呼び出し、ウェアハウス上で推論できることを確認しています。
- 5行中 1行が `-1`（異常）と判定されています。降水量 5.5mm・風速 16.5km/h と **気象条件が他の日より悪い** ことが要因です。

**3. 異常日の詳細（上位10件）**

| カラム | 意味 |
|--------|------|
| `CALL_DROP_RATE` | 切断率（低いほど良好） |
| `PRECIPITATION_MM` | 降水量（mm） |
| `WIND_SPEED_KMH` | 風速（km/h） |
| `ISO_ANOMALY_SCORE` | 異常スコア（**負の値が大きいほど異常度が高い**） |

- スコアが最も低い（異常度が高い）のは **冬季（1月・12月）の降水量0mm + 強風の日** です。  
  乾燥・強風という通常と異なる気象パターンが「孤立しやすい」データとして検出されています。
- 一方、2025年5〜9月には **降水量 60mm超の大雨の日** も異常として検出されています。
- このように IsolationForest は「典型的でないパターン」を多角的に検出できるのが強みです。

### パート B: XGBRegressor でトラフィック予測

**XGBRegressor**（XGBoost の回帰モデル）を使って、トラフィック量を予測します。  
Step 3 の `FORECAST` は時系列専用（ARIMA ベース）でしたが、ここでは **特徴量エンジニアリング** を自分で設計し、  
気象データ・曜日・月などの特徴量を組み合わせた勾配ブースティング回帰モデルを構築します。

#### カスタマイズのポイント

| カスタム要素 | 内容 |
|---|---|
| **ラグ特徴量** | 前日・3日前・7日前のトラフィック量 |
| **移動平均** | 直近7日間の移動平均 |
| **カレンダー特徴量** | 曜日（0-6）、月（1-12） |
| **気象特徴量** | 気温・降水量・風速 |
| **ハイパーパラメータ** | `n_estimators`, `max_depth`, `learning_rate` を調整 |

In [ ]:
# ============================================================
# XGBRegressor: 特徴量エンジニアリング → 訓練 → Registry 登録
# ============================================================
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# --- 特徴量エンジニアリング ---
def add_features(data):
    """ラグ特徴量・移動平均・カレンダー特徴量を追加"""
    d = data.copy()
    d = d.sort_values("MEASURE_DATE")
    # ラグ特徴量
    d["TRAFFIC_LAG_1"]  = d["TOTAL_TRAFFIC_GB"].shift(1)
    d["TRAFFIC_LAG_3"]  = d["TOTAL_TRAFFIC_GB"].shift(3)
    d["TRAFFIC_LAG_7"]  = d["TOTAL_TRAFFIC_GB"].shift(7)
    # 移動平均
    d["TRAFFIC_MA_7"]   = d["TOTAL_TRAFFIC_GB"].rolling(7).mean()
    # カレンダー特徴量
    d["DAY_OF_WEEK"]    = d["MEASURE_DATE"].dt.dayofweek
    d["MONTH"]          = d["MEASURE_DATE"].dt.month
    return d.dropna()

train_fe = add_features(train)
# テストデータ: 訓練データの末尾と連結してラグ特徴量を計算した後に分離
all_data_fe = add_features(pd.concat([train, test], ignore_index=True))
test_fe = all_data_fe[all_data_fe["MEASURE_DATE"] >= "2025-01-01"].copy()

XGB_FEATURES = [
    "TRAFFIC_LAG_1", "TRAFFIC_LAG_3", "TRAFFIC_LAG_7", "TRAFFIC_MA_7",
    "DAY_OF_WEEK", "MONTH",
    "AVG_TEMPERATURE_C", "PRECIPITATION_MM", "WIND_SPEED_KMH"
]
TARGET = "TOTAL_TRAFFIC_GB"

# --- モデル訓練 ---
xgb_model = XGBRegressor(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
xgb_model.fit(
    train_fe[XGB_FEATURES], train_fe[TARGET],
    eval_set=[(test_fe[XGB_FEATURES], test_fe[TARGET])],
    verbose=False
)

# --- 評価 ---
test_fe["XGB_PREDICTED"] = xgb_model.predict(test_fe[XGB_FEATURES])
mae  = mean_absolute_error(test_fe[TARGET], test_fe["XGB_PREDICTED"])
rmse = np.sqrt(mean_squared_error(test_fe[TARGET], test_fe["XGB_PREDICTED"]))
print(f"テストデータ評価: MAE = {mae:.2f} GB, RMSE = {rmse:.2f} GB")

# --- 特徴量の重要度 ---
importance = pd.DataFrame({
    "FEATURE": XGB_FEATURES,
    "IMPORTANCE": xgb_model.feature_importances_
}).sort_values("IMPORTANCE", ascending=False)
print("\n特徴量の重要度:")
print(importance.to_string(index=False))

# --- Model Registry に登録 ---
# Note: パート A と同様、UserWarning が表示されますがエラーではありません。
# 既存モデルがあれば削除して再登録（冪等性を確保）
try:
    reg.delete_model("NQ_TRAFFIC_XGBOOST")
    print("既存モデル NQ_TRAFFIC_XGBOOST を削除しました")
except Exception:
    pass

# target_platforms=["WAREHOUSE"] でウェアハウスからの推論を有効化
xgb_mv = reg.log_model(
    model=xgb_model,
    model_name="NQ_TRAFFIC_XGBOOST",
    version_name="v1",
    conda_dependencies=["xgboost"],
    comment="XGBRegressor: トラフィック予測（ラグ+気象+カレンダー特徴量）",
    sample_input_data=train_fe[XGB_FEATURES],
    target_platforms=["WAREHOUSE"],
    metrics={"MAE": float(mae), "RMSE": float(rmse)}
)

print(f"\n✅ Model Registry に登録完了: NQ_TRAFFIC_XGBOOST v1")

#### 結果の読み方

**1. モデル精度（テストデータ評価）**

```
テストデータ評価: MAE = 19.94 GB, RMSE = 36.78 GB
```

| 指標 | 値 | 意味 |
|------|-----|------|
| **MAE**（Mean Absolute Error） | 19.94 GB | 予測値と実測値の平均的なズレ。トラフィック予測が **平均 ±20GB 程度** の誤差であることを示す |
| **RMSE**（Root Mean Squared Error） | 36.78 GB | 大きな外れ値を重く評価した誤差。MAE より大きいため、**一部の日で大きく外す** ケースがあることを示す |

日次トラフィック量が数百 GB 規模であることを考えると、MAE 約 20GB は実用的な精度です。  
RMSE が MAE の約1.8倍あるため、休日やイベント日など **特異日の予測改善** が精度向上のポイントになります。

**2. 特徴量の重要度**

XGBoost が予測に各特徴量をどれだけ使ったかを示します（合計 = 1.0）。

| 順位 | 特徴量 | 重要度 | 解釈 |
|:----:|--------|--------|------|
| 1 | `DAY_OF_WEEK`（曜日） | **0.383** | トラフィック量を最も左右するのは **曜日**。平日 vs 休日でトラフィックパターンが大きく異なる |
| 2 | `TRAFFIC_LAG_7`（7日前のトラフィック） | **0.363** | 1週間前の同曜日のトラフィックが強い予測因子。**週次の周期性** を捉えている |
| 3 | `TRAFFIC_MA_7`（7日間移動平均） | **0.139** | 直近1週間のトレンドも予測に寄与。急増・急減の傾向を反映 |
| 4 | `TRAFFIC_LAG_1`（前日トラフィック） | 0.056 | 前日の値も参考にされるが、曜日や週次パターンほどではない |
| 5 | `MONTH`（月） | 0.027 | 季節性の影響（夏休み・年末年始など） |
| 6-9 | ラグ3日・気温・風速・降水量 | 各0.001〜0.017 | 気象条件の直接的な影響は小さい |

上位3つ（曜日 + 7日前ラグ + 移動平均）だけで **全体の約88%** を占めています。  
トラフィック量は気象よりも **曜日と週次の周期パターン** に強く依存していることがわかります。

> **改善のヒント**: 祝日フラグやイベント情報を特徴量に追加すると、特異日の予測精度が向上する可能性があります。

In [ ]:
# ============================================================
# Model Registry の確認: 登録したモデル一覧
# ============================================================

# Registry に登録されたモデル一覧
models = reg.show_models()
print("📦 登録済みモデル一覧:")
print(models[["name", "comment"]].to_string(index=False))

# 各モデルのバージョンとメトリクスを確認
for model_name in ["NQ_ISOLATION_FOREST", "NQ_TRAFFIC_XGBOOST"]:
    m = reg.get_model(model_name)
    mv = m.version("v1")
    print(f"\n--- {model_name} v1 ---")
    print(f"  メトリクス: {mv.show_metrics()}")

In [ ]:
# ============================================================
# Registry 経由の推論デモ: 直近30日の異常検知
# ============================================================

# 直近30日のデータを取得
recent_df = session.sql("""
    SELECT
        nq.MEASURE_DATE,
        AVG(nq.CALL_DROP_RATE)          AS CALL_DROP_RATE,
        AVG(nq.AVG_DOWNLOAD_MBPS)       AS AVG_DOWNLOAD_MBPS,
        AVG(nq.AVG_LATENCY_MS)          AS AVG_LATENCY_MS,
        AVG(nq.PACKET_LOSS_RATE)        AS PACKET_LOSS_RATE,
        AVG(w.AVG_RAINFALL)             AS PRECIPITATION_MM,
        AVG(w.AVG_WIND_SPEED)           AS WIND_SPEED_KMH
    FROM TELECOM_AI_HANDSON.ANALYTICS.NETWORK_QUALITY nq
    LEFT JOIN TELECOM_AI_HANDSON.ANALYTICS.DAILY_WEATHER w
        ON nq.MEASURE_DATE = w.WEATHER_DATE
    WHERE nq.MEASURE_DATE >= DATEADD('day', -30, CURRENT_DATE())
    GROUP BY nq.MEASURE_DATE
    ORDER BY nq.MEASURE_DATE DESC
""").to_pandas()
recent_df = recent_df.dropna()

# --- ローカルの iso_model で推論 ---
recent_pred   = iso_model.predict(recent_df[FEATURE_COLS])
recent_scores = iso_model.decision_function(recent_df[FEATURE_COLS])

recent_df["IS_ANOMALY"]     = (recent_pred == -1)
recent_df["ANOMALY_SCORE"]  = recent_scores

anomaly_count = recent_df["IS_ANOMALY"].sum()
print(f"直近30日の異常検知結果: {anomaly_count} 日を異常と判定")

# --- Registry 経由の推論（mv.run → ウェアハウス）---
# Snowpark DataFrame のカラム名をモデル登録時と一致させる
print("\n--- Registry 経由の推論（上位5件）---")
iso_model_ref = reg.get_model("NQ_ISOLATION_FOREST").version("v1")
sample_sp = session.create_dataframe(recent_df[FEATURE_COLS].head(5).reset_index(drop=True))
# create_dataframe は Pandas のカラム名をそのまま使うが、大文字に変換されるため
# モデル登録時の sample_input_data と一致していれば動作する
registry_result = iso_model_ref.run(sample_sp, function_name="predict")
registry_result.show()

# --- 結果表示 ---
recent_df[["MEASURE_DATE", "CALL_DROP_RATE", "PRECIPITATION_MM",
           "WIND_SPEED_KMH", "ANOMALY_SCORE", "IS_ANOMALY"]]

#### 結果の読み方

**1. 直近30日の異常検知サマリー**

```
直近30日の異常検知結果: 9 日を異常と判定
```

- 直近28日間のうち **9 日（約32%）** が異常と判定されました。  
  先ほどのテストデータ全体（13.6%）と比べて異常の割合が高くなっています。  
  これは直近データが訓練データ（2022〜2024年）の傾向からより離れていることを示します。

**2. Registry 経由の推論（上位5件）**

| 日付（推定） | 切断率 | 降水量 | 風速 | 判定 | 注目ポイント |
|---|---|---|---|---|---|
| 2/24 | 0.217 | 0.2mm | 13.8km/h | **正常** | 全指標が平均的 |
| 2/23 | 0.224 | 0.1mm | **27.0km/h** | **異常** | 風速が極端に高い |
| 2/22 | 0.220 | 0.0mm | **19.3km/h** | **異常** | 風速がやや高い |
| 2/21 | **0.372** | 0.0mm | 8.9km/h | **正常** | 切断率が高いが他指標は平均的 |
| 2/20 | 0.211 | 0.0mm | 9.2km/h | **正常** | 全指標が平均的 |

- 2/23 と 2/22 が異常判定された主因は **風速** です（27.0km/h, 19.3km/h）。  
  訓練データの風速分布から大きく外れたデータとして孤立が検出されています。
- 2/21 は切断率 0.372 と高めですが **正常** と判定されています。  
  切断率が周期的に跳ねるパターンは訓練データにも存在するため、「見慣れたパターン」として正常扱いされます。  
  IsolationForest は **単一指標の高低** ではなく **多次元空間での孤立度** で判定することがわかります。

**3. 全30日の詳細テーブル**

| カラム | 意味 |
|--------|------|
| `ANOMALY_SCORE` | 異常スコア。**負の値** = 異常寄り、**正の値** = 正常寄り |
| `IS_ANOMALY` | True = 異常、False = 正常 |

異常と判定された日のパターンは大きく **2種類** に分かれます：

| パターン | 該当日（例） | 特徴 |
|----------|-------------|------|
| **強風** | 2/23（27km/h）, 2/19（24.3km/h）, 2/8（23km/h） | 風速 20km/h 超。切断率は正常範囲 |
| **大雨** | 2/16（10.4mm）, 2/11（9.7mm）, 2/7（11.7mm） | 降水量 10mm 前後。風速も高め |

一方、切断率が 0.37 前後に跳ねる日（2/21, 2/15, 2/6）はスコアが **正の値（正常寄り）** です。  
これは約5日周期で切断率が上がるパターンが訓練データにも含まれており、  
モデルが「よくあるパターン」として学習済みであることを意味します。

> **実運用への示唆**: 切断率の高低だけで監視するよりも、  
> **気象条件を含む多次元の異常スコア** を組み合わせることで、  
> 「通常運用の変動」と「本当に注意すべき異常」を区別できます。

### Step 7 まとめ

OSS ライブラリと Snowflake の **Container Runtime** + **Model Registry** を使って、カスタム ML パイプラインを構築しました。

| モデル | ライブラリ | 用途 | カスタム要素 |
|--------|-----------|------|------------|
| **IsolationForest** | scikit-learn | 異常検知（教師なし） | `contamination` で異常割合を制御、多次元特徴量で検知 |
| **XGBRegressor** | XGBoost | トラフィック予測 | ラグ特徴量・移動平均・カレンダー・気象の特徴量エンジニアリング |

#### ML 開発サイクルの全体像

```
データ取得（Snowpark session.sql → Pandas）
     │
     ▼
特徴量エンジニアリング（Pandas）
     │
     ▼
モデル訓練（scikit-learn / XGBoost）
     │
     ▼
Model Registry に登録（reg.log_model + target_platforms=["WAREHOUSE"]）
     │
     ▼
推論（mv.run で Snowpark DataFrame を渡す → ウェアハウスで実行）
     │
     ▼
Task + Alert で自動化（発展）
```

> **ポイント**: `target_platforms=["WAREHOUSE"]` を指定して登録すると、ウェアハウス上で `mv.run()` による推論が実行できます。  
> **発展**: Model Registry にはバージョン管理機能があるため、モデルを改善したら `v2`, `v3` と登録し、  
> `DEFAULT_VERSION` を切り替えることで **ダウンタイムなし** でモデルを更新できます。